model evaluation of the first bert model.

what do I need?

- the test data labelled by the bert model
- the human labels, arranged in the same order as in the test label.

for two above, since we don't have date of creation, we'll use IDs to arrange the rows

In [1]:
import pandas as pd
import os
from glob import glob
import ast
import re
import spacy
import time

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, adjusted_rand_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Access dataset files
folder_path = os.path.join( "..", "Data") 
file_type = "*.csv"

# List of dataset file paths
document_path = glob(os.path.join(folder_path, file_type))

document_path

['../Data/bert_default_raw_text_topics_term.csv',
 '../Data/subset_sample_no_label.csv',
 '../Data/test_data_bert_raw_label.csv',
 '../Data/bert_train_lemma.csv',
 '../Data/human_and_bert_raw_labels.csv',
 '../Data/bert_kmeans_lemma_topics_terms.csv',
 '../Data/filtered_corpus.csv',
 '../Data/test_data_human_label_bert_raw.csv',
 '../Data/full_posts.csv',
 '../Data/full_data_no_preprocessing.csv',
 '../Data/training_data.csv',
 '../Data/lda_train.csv',
 '../Data/vocabulary.csv',
 '../Data/bert_train_no_prep.csv',
 '../Data/emoji_subset.csv',
 '../Data/sampled_subset.csv',
 '../Data/subs_topics.csv',
 '../Data/comments.csv']

In [16]:
#import cleaned data

def list_converter(text):
    #to revert list->str conversion from pd.read_csv
    return ast.literal_eval(text)


model_labelled_data = pd.read_csv("../Data/test_data_bert_raw_label.csv", converters ={'tokens':list_converter})
human_labelled_data = pd.read_csv("../Data/test_data_human_label_bert_raw.csv", converters ={'tokens':list_converter})
#test_data = test_data.drop(columns = ['index', 'clean_text', 'tokens', 'word_count'])

In [17]:
model_labelled_data.drop(columns = ['Unnamed: 0'], inplace = True)
model_labelled_data.head()

,text_type,ID,date_created,long_text,topic_description
0,submission,wbwik,2012-07-10 19:11:08,Only in Dubai! R8: BRAND NEW 2012 Audi R8 V8 -...,1_dubai_uae_arabic_people
1,comment,c5c2v3q,2012-07-10 22:36:50,"Surprise! Happy birthday! \n\nAh man, *anoth...",-1_dubai_people_just_like
2,comment,c5cc9sz,2012-07-11 08:48:34,I looked at the AD and it seems more like a sa...,22_news_ad_gn_ads
3,comment,c5cdavh,2012-07-11 10:15:52,It's a marketing post. But isn't it against du...,-1_dubai_people_just_like
4,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],1_dubai_uae_arabic_people


In [ ]:
model_labelled_data.sort_values?

In [18]:
#split `topic_description` into two columns for topic number and topic description
model_labelled_data[['model_topic_no', 'model_topic_desc']] = model_labelled_data['topic_description'].str.split('_', n = 1, expand = True)
model_labelled_data['model_topic_no'] = model_labelled_data['model_topic_no'].astype(int) 
model_labelled_data.drop(columns = ['topic_description'], inplace = True)

In [19]:
#sort by `ID`
model_labelled_data.sort_values(by=['ID'], ignore_index = True, inplace = True)
model_labelled_data.head()

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc
0,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],1,dubai_uae_arabic_people
1,submission,13ch705,2023-05-09 10:04:19,Anyone here have any idea where I can park my ...,10,lane_speed_left_parking
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",1,dubai_uae_arabic_people
3,submission,13p1nlq,2023-05-23 00:07:39,A Well Deserved Shout-out I posted in this sub...,7,talabat_delivery_price_zomato
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,24,rent_landlord_property_estate


In [20]:
model_labelled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text_type         1200 non-null   object
 1   ID                1200 non-null   object
 2   date_created      1200 non-null   object
 3   long_text         1200 non-null   object
 4   model_topic_no    1200 non-null   int64 
 5   model_topic_desc  1200 non-null   object
dtypes: int64(1), object(5)
memory usage: 56.4+ KB


In [21]:
#dataset showing the human labels
human_labelled_data.head()

,ID,long_text,topics,topic_number_bert_raw
0,10embj,Dubai cap my dad brought me! [FYSR],undefined,-1
1,13ch705,Anyone here have any idea where I can park my ...,infrastructure.transportation,32
2,13cpma4,"UAE real estate agents and brokers, I need you...",real_estate_housing,24
3,13p1nlq,A Well Deserved Shout-out I posted in this sub...,shopping,30
4,14d7052,Furnished apartments‚Ä¶ bedding/pots and pans ...,real_estate_housing,24


In [22]:
#include column in model_labelled_data to hold the human topic numbers

model_labelled_data[['human_topic_desc','human_topic_no']] = human_labelled_data[['topics','topic_number_bert_raw']].copy()

In [23]:
model_labelled_data.head()

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc,human_topic_desc,human_topic_no
0,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],1,dubai_uae_arabic_people,undefined,-1
1,submission,13ch705,2023-05-09 10:04:19,Anyone here have any idea where I can park my ...,10,lane_speed_left_parking,infrastructure.transportation,32
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",1,dubai_uae_arabic_people,real_estate_housing,24
3,submission,13p1nlq,2023-05-23 00:07:39,A Well Deserved Shout-out I posted in this sub...,7,talabat_delivery_price_zomato,shopping,30
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,24,rent_landlord_property_estate,real_estate_housing,24


In [24]:
model_labelled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text_type         1200 non-null   object
 1   ID                1200 non-null   object
 2   date_created      1200 non-null   object
 3   long_text         1200 non-null   object
 4   model_topic_no    1200 non-null   int64 
 5   model_topic_desc  1200 non-null   object
 6   human_topic_desc  1200 non-null   object
 7   human_topic_no    1200 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 75.1+ KB


In [25]:
model_labelled_data.to_csv('../Data/human_and_bert_raw_labels.csv')

<h2> ACCURACY SCORE </h2>

<h3>Accuracy score of the entire model prediction</h3>

In [37]:
#accuracy score
y_true = model_labelled_data.human_topic_no
y_pred = model_labelled_data.model_topic_no
accuracy = accuracy_score(y_true, y_pred, normalize = True)

print (f'Bertopic model with default parameters and trained on unprocessed data\nModel accuracy is: {accuracy}')

Bertopic model with default parameters and trained on unprocessed data
Model accuracy is: 0.365


<h3>Accuracy score of the individual human labels</h3>

In [77]:
topic_range = sorted(model_labelled_data['human_topic_no'].unique())
accuracy = {'topic_no':[],
            'human_topic_desc':[],
           'accuracy_score':[]
           }

for topic_no in topic_range:
    accuracy['topic_no'].append(topic_no) 
    mask = model_labelled_data.human_topic_no == topic_no
    topic_df = model_labelled_data[mask]
    
    accuracy['human_topic_desc'].append(topic_df['human_topic_desc'].unique())
    
    #accuracy
    y_true = topic_df.human_topic_no
    y_pred = topic_df.model_topic_no
    acc = accuracy_score(y_true, y_pred, normalize = True)
    
    accuracy['accuracy_score'].append(acc)

    
accuracy_df = pd.DataFrame(accuracy)

accuracy_df

,topic_no,human_topic_desc,accuracy_score
0,-2,[unrelated_theme],0.000000
1,-1,[undefined],0.541667
2,0,[reddit_usage],0.361111
3,1,[uae_regions],0.600000
4,3,"[diversity_inclusion.race/nationality, diversi...",0.200000
...,...,...,...
64,74,[religious_tolerance],0.000000
65,75,[safety.personal_safety],0.000000
66,76,[supernatural_activities],0.000000
67,77,[travelling],0.000000


In [75]:
accuracy_df.set_index('topic_no', inplace = True)
accuracy_df.sort_values(by ='accuracy_score', ascending = False)

,human_topic_desc,accuracy_score
topic_no,,
35,"[diversity_inclusion.sexuality, diversity & in...",1.0
46,[communication_etiquette],1.0
41,[personal_habits.smoking],1.0
29,[beverages],1.0
11,[animal_welfare],1.0
...,...,...
49,[adult_entertainment],0.0
48,[dubai_urban_centers],0.0
44,[arabic_language],0.0


In [76]:
accuracy_df[accuracy_df.accuracy_score >=0.5]

,human_topic_desc,accuracy_score
topic_no,,
-1,[undefined],0.541667
1,[uae_regions],0.600000
6,[weather_conditions],0.700000
11,[animal_welfare],1.000000
15,[infrastructure.communication_internet],0.687500
18,[immigration],0.636364
19,[vehicles],0.555556
20,[social relationships.family],0.500000
21,[islam_religion],0.692308


In [39]:
mask = model_labelled_data.human_topic_no == 35
model_labelled_data[mask]

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc,human_topic_desc,human_topic_no
1155,comment,jo0ebba,2023-06-13 21:04:33,"Not everyone has to support this ""pride"" thing...",35,gay_trans_lgbt_homosexuality,diversity_inclusion.sexuality,35
1162,comment,jo3xdhg,2023-06-14 18:49:48,"I understand your point, but I want to share a...",35,gay_trans_lgbt_homosexuality,diversity & inclusion.sexuality,35


In [ ]:
#confusion matrix
y_true = topic_df['human_label']#model_labelled_data['human_label']
y_pred = topic_df ['topic_no'] #model_labelled_data['topic_no']
labels = sorted(list(set(y_true)))
cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
print (cm)

<h2> PRECISION, RECALL, F1-SCORE </h2>

In [53]:
#accuracy score
y_true = model_labelled_data.human_topic_no
y_pred = model_labelled_data.model_topic_no
#labels = list(model_labelled_data['human_topic_no'].unique())
clr = classification_report(y_true, y_pred,zero_division = 0.0, output_dict = True)

In [61]:
clr_df = pd.DataFrame(clr).T
clr_df

,precision,recall,f1-score,support
-2,0.000000,0.000000,0.000000,26.000
-1,0.393939,0.541667,0.456140,432.000
0,0.175676,0.361111,0.236364,36.000
1,0.071429,0.600000,0.127660,10.000
2,0.000000,0.000000,0.000000,0.000
...,...,...,...,...
77,0.000000,0.000000,0.000000,12.000
78,0.000000,0.000000,0.000000,1.000
accuracy,0.365000,0.365000,0.365000,0.365
macro avg,0.232781,0.242637,0.211381,1200.000
